In [14]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
import csv
import os
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

NameError: name 'framework' is not defined

In [5]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic
mp_drawing_styles = mp.solutions.drawing_styles

NameError: name 'mp' is not defined

In [6]:
def calculate_angle(a,b):
    a = np.array(a)
    b = np.array(b)
    
    radians = np.arctan2(b[1]-a[1], b[0]-a[0])
    angle = np.abs(radians*180.0/np.pi)

    return angle 

In [10]:
cap = cv2.VideoCapture('./mhchoi.mp4')
anomaly = {'shoulder':[],'hand':[]}

features =['left_wrist','left_pinky','left_index','left_thumb',
'right_wrist','right_pinky','right_index','right_thumb']

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    start_time = datetime.now()
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break 
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # # 1. Draw face landmarks
        # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
        #                         mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
        #                         mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
        #                         )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                )
        # Export coordinates
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get shoulder coordinates
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y]
            
            # Calculate shoulder angle
            angle = calculate_angle(left_shoulder, right_shoulder)
            if angle<=170: # 기준 정해야함
                anomaly['shoulder'].append(time.time())
            else:
                shoulder_seconds = anomaly["shoulder"][-1]-anomaly["shoulder"][0]
                if shoulder_seconds >= 1e-3:
                    print(f'{shoulder_seconds:.3f}초 동안 자세가 좋지 않았습니다.')
                anomaly['shoulder']=[]

            if results.left_hand_landmarks or results.right_hand_landmarks:
                current_time = datetime.now()
                target_time = current_time - start_time
                print(f'{target_time.total_seconds():.3f}초에 손이 나왔습니다.')


        except:
            pass
                        
        cv2.imshow('Video Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)

NameError: name 'cv2' is not defined

In [68]:
landmarks[mp_pose.PoseLandmark['LEFT_WRIST']]

x: 0.9589765667915344
y: 1.5836498737335205
z: -0.9355209469795227
visibility: 0.08272692561149597

In [104]:
anomaly['hand']

[time.struct_time(tm_year=2023, tm_mon=1, tm_mday=14, tm_hour=2, tm_min=39, tm_sec=17, tm_wday=5, tm_yday=14, tm_isdst=0),
 time.struct_time(tm_year=2023, tm_mon=1, tm_mday=14, tm_hour=2, tm_min=39, tm_sec=17, tm_wday=5, tm_yday=14, tm_isdst=0),
 time.struct_time(tm_year=2023, tm_mon=1, tm_mday=14, tm_hour=2, tm_min=39, tm_sec=17, tm_wday=5, tm_yday=14, tm_isdst=0),
 time.struct_time(tm_year=2023, tm_mon=1, tm_mday=14, tm_hour=2, tm_min=39, tm_sec=18, tm_wday=5, tm_yday=14, tm_isdst=0),
 time.struct_time(tm_year=2023, tm_mon=1, tm_mday=14, tm_hour=2, tm_min=39, tm_sec=18, tm_wday=5, tm_yday=14, tm_isdst=0),
 time.struct_time(tm_year=2023, tm_mon=1, tm_mday=14, tm_hour=2, tm_min=39, tm_sec=19, tm_wday=5, tm_yday=14, tm_isdst=0),
 time.struct_time(tm_year=2023, tm_mon=1, tm_mday=14, tm_hour=2, tm_min=39, tm_sec=19, tm_wday=5, tm_yday=14, tm_isdst=0),
 time.struct_time(tm_year=2023, tm_mon=1, tm_mday=14, tm_hour=2, tm_min=39, tm_sec=19, tm_wday=5, tm_yday=14, tm_isdst=0),
 time.struct_tim